# INSTRUCTIONS
1. Create a virtual environment
2. Install the required packages provided in the requirements.txt file
    > pip install requirements.txt
3. Download & Install npcap from the following link "https://npcap.com/dist/npcap-1.80.exe"
4. Ensure that you run the notebook with admin priviledges.
5. The result is a csv file named "network_scan_report.csv" automatically created in the same path as where the notebook was run.

## Import required libraries and packages

In [38]:
# Import required packages from the Scapy library
import scapy.all as scapy 
from scapy.all import ARP, Ether, srp, conf
import csv
import socket
import ipaddress

## Confirm if npcap is well configured

In [39]:
print(conf.ifaces)

Source   Index  Name                                  MAC                    IPv4             IPv6                     
libpcap  1      Software Loopback Interface 1         00:00:00:00:00:00      127.0.0.1        ::1                      
libpcap  10     Realtek PCIe GbE Family Controller    HP:d5:f6:37            192.168.62.10    fe80::b44:3353:b158:8394 
libpcap  14     Microsoft Wi-Fi Direct Virtual Adap_  ba:d5:7a:a7:82:e5      169.254.166.134  fe80::81a4:b229:e8a9:ca18
libpcap  16     WAN Miniport (IP)                                                                                      
libpcap  19     Realtek RTL8821CE 802.11ac PCIe Ada_  CloudNetwork:a7:82:e5  169.254.101.1    fe80::2f6f:589d:511f:5a28
libpcap  4      WAN Miniport (Network Monitor)                                                                         
libpcap  44     Hyper-V Virtual Ethernet Adapter      Microsoft:f4:c6:15     172.19.144.1     fe80::9536:760b:dde:157a 
libpcap  8      WAN Miniport (IPv6)     

## Get the local IP Address of your device

In [40]:
def getLocalIp():
    # Gets the local IP address of the machine
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        # Connect to Google's DNS server
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        print(local_ip)
        return local_ip
    except Exception as e:
        print(f"Error getting local IP: {e}")
        return None

getLocalIp()

192.168.62.10


'192.168.62.10'

## Get the network range

In [41]:
def getNetworkRange(ipAddress):
    # Gets the network range based on the Ip Address
    if not ipAddress:
        return None
    try:
        network = ipaddress.ip_network(ipAddress + '/24', strict=False)
        print(network)
        return str(network)
    except Exception as e:
        print(f"Error getting network range: {e}")
        return None

# getNetworkRange('192.168.62.10')

## Function to scan network and get the connected devices

In [42]:
def scanNetwork(network):
    print("scanning network", network)
    # Create an ARP request
    arp = ARP(pdst=network)
    ether = Ether(dst="ff:ff:ff:ff:ff:ff")
    packet = ether/arp

    # Send the packet and capture the responses
    # conf.L3socket = L3RawSocket
    result = srp(packet, timeout=3, verbose=0)[0]

    devices = []
    for sent, received in result:
        devices.append({'IP': received.psrc, 'MAC': received.hwsrc})
         
    print(devices)
    return devices
    
# scanNetwork("192.168.62.0/24")

## Save devices found to a csv

In [43]:
def saveToCsv(devices, filename):
    # Saves the scanned devices to a CSV file.
    with open(filename, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['IP', 'MAC'])
        writer.writeheader()
        for device in devices:
            writer.writerow(device)

## Execute all the functions from the block below.

In [44]:
def scan():
    # Get IP Address for my network
    ipAddress = getLocalIp()
    
    # Return if no IP found
    if not ipAddress:
        print("Failed to get local IP. Exiting.")
        return
    
    # Get the network range
    networkRange = getNetworkRange(ipAddress)

    # Scan the network using the network range found
    print("Scanning the network...")
    devices = scanNetwork(networkRange)
    if devices:
        print(f"Found {len(devices)} devices on the network.")
        for device in devices:
            print(f"IP: {device['IP']}, MAC: {device['MAC']}")
        # Save results to a CSV file
        filename = "network_scan_report.csv"
        saveToCsv(devices, filename)
        print(f"Report saved to {filename}.")
    else:
        print("No devices found on the network.")

scan()

192.168.62.10
192.168.62.0/24
Scanning the network...
scanning network 192.168.62.0/24
[{'IP': '192.168.62.1', 'MAC': '90:6c:ac:99:4c:0c'}, {'IP': '192.168.62.10', 'MAC': 'c0:18:03:d5:f6:37'}, {'IP': '192.168.62.16', 'MAC': '48:0f:cf:3b:fd:17'}, {'IP': '192.168.62.25', 'MAC': '78:04:e3:96:21:79'}, {'IP': '192.168.62.26', 'MAC': '48:0f:cf:66:91:b1'}, {'IP': '192.168.62.30', 'MAC': 'dc:4a:3e:7f:b9:ba'}, {'IP': '192.168.62.31', 'MAC': '78:04:e3:96:2c:79'}, {'IP': '192.168.62.32', 'MAC': '78:04:e3:96:26:d6'}, {'IP': '192.168.62.36', 'MAC': '78:04:e3:95:fa:03'}, {'IP': '192.168.62.41', 'MAC': '48:0f:cf:45:38:a3'}, {'IP': '192.168.62.35', 'MAC': 'e8:65:d4:20:2c:30'}, {'IP': '192.168.62.64', 'MAC': '78:04:e3:96:1b:57'}, {'IP': '192.168.62.65', 'MAC': '48:8f:5a:9b:23:82'}, {'IP': '192.168.62.66', 'MAC': 'a0:8c:fd:c3:8c:9d'}, {'IP': '192.168.62.69', 'MAC': 'c4:65:16:24:ae:b1'}, {'IP': '192.168.62.74', 'MAC': '48:0f:cf:3b:fc:f9'}, {'IP': '192.168.62.76', 'MAC': '50:0f:f5:35:94:80'}, {'IP': '192.